In [ ]:
from pathlib import Path
import requests
import zipfile
import numpy as np
import pandas as pd
import csv
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from etl.sources.census.etl_utils import get_state_fips_codes

DATA_PATH = Path.cwd().parent / "data"
CSV_PATH = data_path / "dataset" / "ejscreen_2020"

In [ ]:
download = requests.get(
    "https://gaftp.epa.gov/EJSCREEN/2020/EJSCREEN_2020_StatePctile.csv.zip",
    verify=False,
)
file_contents = download.content
zip_file_path = DATA_PATH / "tmp"
zip_file = open(zip_file_path / "downloaded.zip", "wb")
zip_file.write(file_contents)
zip_file.close()

In [ ]:
with zipfile.ZipFile(zip_file_path / "downloaded.zip", "r") as zip_ref:
    zip_ref.extractall(zip_file_path)
ejscreen_csv = data_path / "tmp" / "EJSCREEN_2020_StatePctile.csv"

In [ ]:
df = pd.read_csv(ejscreen_csv, dtype={"ID": "string"}, low_memory=False)

In [ ]:
# write nationwide csv
csv_path.mkdir(parents=True, exist_ok=True)
df.to_csv(CSV_PATH / f"usa.csv", index=False)

In [ ]:
# write per state csvs
for fips in get_state_fips_codes(DATA_PATH):
    print(f"Generating data{fips} csv")
    df1 = df[df.ID.str[:2] == fips]
    # we need to name the file data01.csv for ogr2ogr csv merge to work
    df1.to_csv(CSV_PATH / f"data{fips}.csv", index=False)